# ROB2004 Final Project - pick and place objects

The goal of this project is to solve a simple manipulation task: picking up objects and moving them in a bowl.

## Instructions
* Answer all questions in the notebook
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a 2-3 pages report in pdf format (pdf only) detailing the methodology you followed to answer the questions as well as answers to the questions that require a typeset answer. You may add the plots in the report (does not count for the page limit) or in the Jupyter notebook.
* This is an individual project

We will use a model of the Frank-Emika Panda robot. This robot has 7 revolute joints and its kinematics is described in the picture below.

You will need to install pybullet on your computer. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./panda.jpg" width="200"/>
</div>

# Question

Implement a controller using the code skeletong below in order to get the robot to go and pick up the red blocks and drop them in the green bowl. The bowl position is $(-0.3,0.55,0.65)$ (in spatial frame coordinates) and the blocks positions are $(0.35,0.58,0.65)$ and $(0.15,0.67,0.65)$ respectively.

<div>
<img src="./manipulation_scene.jpg" width="300"/>
</div>

You are free to use the methods that you want to solve the task, with the following constraints:
* You cannot use any external library apart from numpy and scipy
* You need to use at least one controller in the end-effector space
* You need to generate smooth motions
* You need to compensate for the gravity of the robot

In your report, describe the controller and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, velocities, joint trajectories, etc as you see fit).

In [1]:
import numpy as np
import time

import matplotlib.pyplot as plt

from panda import Simulator, PandaRobot

np.set_printoptions(precision=4, suppress = True)

pybullet build time: Aug 21 2022 18:46:39


# Helper functions

We provide a set of helper functions (forward kinematics, Jacobians, gravity terms) that can be used to implement the desired controllers. These functions are implemented in the PandaRobot class (in the panda.py file).
Examples are shown below:

In [2]:
# create an instance of the helper class
my_robot = PandaRobot()

# compute the forward kinematics for an arbitrary joint configuration
q = np.random.uniform(-1,1,7)
pose = my_robot.FK(np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,np.pi/4]))
# pose = my_robot.FK(q)
print(f'the pose of the end-effector for joint configuration\n\n {q}\n\n is\n\n {pose}')

the pose of the end-effector for joint configuration

 [ 0.963   0.2494 -0.6168  0.9772 -0.0434 -0.9924  0.2783]

 is

 [[ 0.      1.      0.      0.    ]
 [ 1.     -0.     -0.      0.5545]
 [ 0.      0.     -1.      0.6245]
 [ 0.      0.      0.      1.    ]]


In [3]:
# we can also get the Jacobian in various frames 
# it is sufficient to pass the name of the frame as an argument as shown below

q = np.random.uniform(-1,1,7)

spatial_jacobian = my_robot.get_jacobian(q, 'S')
body_jacobian = my_robot.get_jacobian(q, 'B')

# we can even get the Jacobian in a frame place at the same position as the end-effector 
# frame but oriented like the spatial frame
oriented_jacobian = my_robot.get_jacobian(q, 'O')

print(f'for joint configuration \n\n {q} \n \n the spatial Jacobian is \n\n {spatial_jacobian}\n\n')
print(f'the body Jacobian is \n\n {body_jacobian}\n\n')
print(f'and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is\n\n {oriented_jacobian}')

for joint configuration 

 [-0.1063  0.0768 -0.2468 -0.2355 -0.2133  0.3717  0.8208] 
 
 the spatial Jacobian is 

 [[ 0.     -0.9944  0.0081  0.9384  0.0885  0.8464  0.1082]
 [ 0.      0.1061  0.0763 -0.3451  0.2925 -0.5261  0.0175]
 [ 1.      0.      0.9971  0.0188  0.9522  0.0829 -0.994 ]
 [ 0.     -0.0353 -0.0254  0.2234 -0.1687  0.5549 -0.2318]
 [ 0.     -0.3311  0.0027  0.6018  0.0557  0.871   0.1954]
 [ 0.      0.      0.     -0.1058 -0.0014 -0.1383 -0.0218]]


the body Jacobian is 

 [[ 0.0746 -0.4598  0.1423  0.2317  0.363   0.0353 -0.    ]
 [ 0.0803 -0.8817  0.0444  0.9698 -0.0128  0.9994  0.    ]
 [-0.994  -0.1058 -0.9888  0.0769 -0.9317  0.      1.    ]
 [-0.0405  0.5138 -0.0196 -0.2725 -0.0043  0.1069  0.    ]
 [-0.2321 -0.2969 -0.1913  0.0766 -0.1208 -0.0038 -0.    ]
 [-0.0218  0.2416 -0.0114 -0.1454 -0.     -0.088  -0.    ]]


and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is

 [[ 0.     -0.9944  0.0081  0.

In [4]:
# finally it is possible to compute the effect of gravity on the joints
# this can for example be used to do gravity compensation when controlling the robot

q = np.random.uniform(-1,1,7)

g = my_robot.g(q)

print(f'for joint configuration \n\n {q} \n \n the gravity force seen on the joints is \n \n {g}')

for joint configuration 

 [ 0.7641  0.3445 -0.0101 -0.2062  0.5693 -0.9586  0.9662] 
 
 the gravity force seen on the joints is 
 
 [  0.     -25.0894  -0.0265   7.1683  -0.0066  -0.555    0.    ]


# Code Skeleton for the simulation

Feel free to change the `run_time` variable to match your needs. The class `Simulator` contains all the code for the simulation. The code skeleton currently implement a simple PD controller in joint space that just keeps a desired current position.

Importantly you can control both the joints and the gripper of the robot:
* `simulator.send_joint_torque(joint_torques)` sends a vector of dimension 7 setting the torques of the joints
* `simulator.gripper_move(gripper_position)` sends a vector of dimension 2 setting the position of the fingers of the gripper. The fingers can have positions from 0 to 0.04cm (you can fully close the gripper by setting `simulator.gripper_move([0,0])` and fully open it by setting `simulator.gripper_move([0.04,0.04)`)

In [5]:
def compute_trajectory(position_init, position_goal, t_init, t_goal, t):
    ''' Parameter can be angle or coordinates, 5th order'''
    T = t_goal - t_init
    dt = t - t_init
    dpos = position_goal - position_init
    pos_param = 10 * (dt / T) ** 3 - 15 * (dt / T) ** 4 + 6 * (dt / T) ** 5
    vel_param = (30 * (dt / T) ** 2 - 60 * (dt / T) ** 3 + 30 * (dt / T) ** 4) / T
    desired_position = position_init + pos_param * dpos
    desired_velocity = vel_param * dpos
    
    # we return the answer
    return desired_position, desired_velocity

In [6]:
# here we create a simulation and reset the state of the robot
simulator = Simulator()
simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,np.pi/4])

# we create a robot object so we can use its helper functions
my_robot = PandaRobot()

# duration of the simulation
run_time = 60.

# simulation time step
dt = 0.005
num_steps = int(run_time/dt)


# we store information
ndofs = 7 # number of degrees of freedom (excluding the gripper)
measured_joint_pos = np.zeros([num_steps,ndofs])
measured_joint_vel = np.zeros_like(measured_joint_pos)
desired_torques = np.zeros_like(measured_joint_pos)
desired_joint_pos = np.zeros_like(measured_joint_pos)
desired_joint_vel = np.zeros_like(measured_joint_pos)
time = np.zeros([num_steps])

'''Record EE Coord'''
measured_ee_pos = np.zeros([num_steps,3])
measured_ee_vel = np.zeros_like(measured_ee_pos)

### this is the main control loop
for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = simulator.get_state()
    measured_joint_pos[i,:] = q
    measured_joint_vel[i,:] = dq
    pose = my_robot.FK(q)
    measured_ee_pos[i,:] = pose[0:3,3]
    if i > 0: # Get rid of first iter
        measured_ee_vel[i,:] = (measured_ee_pos[i,:] - measured_ee_pos[i-1,:])/dt
    
    ## controller: TODO HERE IMPLEMENT YOUR CONTROLLER TO SOLVE THE TASK
    ## you will need to replace the naive PD controller implemented below

    origin = np.array([0.35,0.58,0.65])
    block1up = np.array([0.35,0.58,0.65])
    block2up = np.array([0.15,0.67,0.65])
    # desired joint positions and velocities
    
    if i > 1000:
        q_des = np.array([0.,0,0.0,-np.pi/2.,0.,np.pi- q[1] + q[3],q[0]+np.pi/4])
    else:
        q_des = np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,np.pi/4])
#     q_des = np.array([0.,0,0.0,-np.pi/2.,0.,np.pi- q[1] + q[3],q[0]+np.pi/4])
    dq_des = np.zeros([7])
    
    # we save the desired positions/velocities for later plotting 
    desired_joint_pos[i,:] = q_des
    desired_joint_vel[i,:] = dq_des
    
    ##PD controller
    # the PD gains
    P = np.array([100., 100., 100., 100., 100., 100., 50.])
    D = np.array([10.,10,10,10,10,10,1.])

    error = q_des - q # the position error for all the joints
    d_error = dq_des-dq # the velocity error for all the joints
    
    K = np.array([300,300,300])
    B = np.array([30,30,30])
    jacob = my_robot.get_jacobian(q, 'O')[3:,:]
    # we compute the desired torques as a PD controller
#     if i > 1000:
    joint_torques = jacob.T @ (np.diag(K) @ (origin - measured_ee_pos[i,:]) - np.diag(B) @ (measured_ee_vel[i,:]))
    joint_torques[2] = 100 * error[2] + 10* d_error[2]
    joint_torques[4] = 100 * error[4] + 10* d_error[4]
    joint_torques[5] = 100 * error[5] + 10* d_error[5]
    joint_torques[6] = 100 * error[6] + 10* d_error[6]
#     else:
#         joint_torques = np.diag(P) @ error + np.diag(D) @ d_error 
    
    
    
    joint_torques += my_robot.g(q)
# joint 5 must be vertical
#     qdes_5 = q[1] + q[3]
# joint 6 must be parallel to edge
#     joint_torques[6] = -joint_torques[0]
#     joint_torques += my_robot.g(q)
    if i % 100 == 0:
#         print(q[1], q[3], q[5], q_des[5])
        print(pose)
    
    # we start by closing the gripper and then open it after 5 seconds
    gripper_position = [0,0]
    if time[i] > 5.0:
        gripper_position = [0.04, 0.04]
    
    # we send them to the robot and do one simulation step
    simulator.send_joint_torque(joint_torques)
    simulator.gripper_move(gripper_position)
    simulator.step()

[[ 0.      1.      0.      0.    ]
 [ 1.     -0.     -0.      0.5545]
 [ 0.      0.     -1.      0.6245]
 [ 0.      0.      0.      1.    ]]
[[ 0.7059  0.6998  0.1092  0.4421]
 [ 0.6957 -0.714   0.0783  0.5006]
 [ 0.1328  0.0207 -0.9909  0.6323]
 [ 0.      0.      0.      1.    ]]
[[ 0.4473  0.8686  0.2131  0.3581]
 [ 0.8084 -0.4946  0.319   0.6288]
 [ 0.3825  0.0296 -0.9235  0.6624]
 [ 0.      0.      0.      1.    ]]
[[ 0.4369  0.8753  0.2074  0.3615]
 [ 0.747  -0.4815  0.4583  0.6245]
 [ 0.501  -0.0453 -0.8642  0.6765]
 [ 0.      0.      0.      1.    ]]
[[ 0.4567  0.836   0.3043  0.3863]
 [ 0.7277 -0.5478  0.4128  0.6121]
 [ 0.5117  0.0329 -0.8585  0.6673]
 [ 0.      0.      0.      1.    ]]
[[ 0.4344  0.8663  0.2466  0.3662]
 [ 0.7364 -0.4992  0.4567  0.6223]
 [ 0.5187 -0.0168 -0.8548  0.6716]
 [ 0.      0.      0.      1.    ]]
[[ 0.4451  0.8524  0.2745  0.3759]
 [ 0.7317 -0.5229  0.4373  0.6173]
 [ 0.5163  0.0062 -0.8564  0.6701]
 [ 0.      0.      0.      1.    ]]
[[ 0.442   0.

KeyboardInterrupt: 